In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm, chi2
from joblib import Parallel, delayed
from tqdm import tqdm
from scipy.interpolate import interp1d
import itertools

# -----------------------------
# ODE model
# -----------------------------
def ode_rhs(t, y, p):
    a1, a2, a3, a4, a5, a6 = p
    y1, y2 = y
    dy1 = a1 * y1 * (1 - y1 / a2) - a3 * y1 * y2
    dy2 = a4 - a5 * y2 - a6 * y1 * y2
    return [dy1, dy2]

def solve_dataset(t_obs, params, y10, y20, method="BDF", rtol=1e-5, atol=1e-7):
    sol = solve_ivp(lambda t, y: ode_rhs(t, y, params),
                    (t_obs[0], t_obs[-1]), [y10, y20],
                    t_eval=t_obs, method=method, rtol=rtol, atol=atol)
    return sol.y[0] + sol.y[1]

# -----------------------------
# Experimental setup
# -----------------------------
t_obs = [
    np.array([14, 16, 18, 20, 22, 24, 26, 28, 30], float),
    np.array([12, 14, 16, 18, 20, 22, 24, 26], float),
    np.array([ 6,  8, 10, 12, 14, 16, 18, 20], float),
    np.array([ 2,  4,  6,  8, 10, 12, 14], float)
]

theta_true = np.array([
    0.380159, 1.8637e+09, 1.40997e-08,
    125102,   0.00308406, 3.33798e-09,
    1.43323e+06,
    9.38924e+06, 2.5943e+07, 2.48114e+07, 6.61559e+07
])

lb_kin = np.array([0.35, 1.6e9, 0.8e-8, 8e4, 1e-3, 1e-9], float)
ub_kin = np.array([0.38, 2.2e9, 2.7e-8, 1.4e5, 1.8e-1, 5.5e-9], float)
y1min, y1max = 1e3, 5e8
y2min, y2max = 1e1, 1e7
lb_all = np.concatenate([lb_kin, [y2min], [y1min]*4])
ub_all = np.concatenate([ub_kin, [y2max], [y1max]*4])
bounds_list = list(zip(lb_all, ub_all))

# -----------------------------
# Generate noisy data
# -----------------------------
CV, SigmaFloor = 0.025, 1e5
exp_data, sigma_data = [], []
params_true = theta_true[:6]
y2_true = theta_true[6]
y1_trues = theta_true[7:]

for i, y10 in enumerate(y1_trues):
    clean = solve_dataset(t_obs[i], params_true, y10, y2_true, "BDF")
    sigma_vals = np.maximum(CV * clean, SigmaFloor)
    noisy = clean + np.random.normal(0, sigma_vals)
    exp_data.append(noisy)
    sigma_data.append(sigma_vals)

print("Synthetic noisy data generated.")

# -----------------------------
# Log-likelihood
# -----------------------------
def loglikelihood(theta, method="BDF"):
    a_params = theta[:6]
    y20 = theta[6]
    y10s = theta[7:]
    ll_total = 0.0
    for i in range(4):
        pred = solve_dataset(t_obs[i], a_params, y10s[i], y20, method, rtol=1e-5, atol=1e-7)
        resid = exp_data[i] - pred
        sigmas = sigma_data[i]
        ll_total += np.sum(norm.logpdf(resid, loc=0, scale=sigmas))
    return ll_total

def neg_ll(theta):
    return -loglikelihood(theta, method="BDF")

# -----------------------------
# Compute MLE with global + local search (faster settings)
# -----------------------------
print("Running global optimization...")
res_global = differential_evolution(neg_ll, bounds_list, maxiter=20, popsize=10, polish=False, workers=-1, tol=0.1)
print("Global search done. Refining with local search...")
res_local = minimize(neg_ll, res_global.x, bounds=bounds_list,
                     method="L-BFGS-B", options={"maxiter":100, "ftol":1e-5})
theta_mle = res_local.x
print("Optimization success:", res_local.success)
print("MLE parameters:", theta_mle)

# -----------------------------
# Profile likelihood with warm starts (only for kinetic parameters)
# -----------------------------
def profile_likelihood(param_index, theta_mle, bounds, method="BDF", n_grid=15):
    p_grid = np.linspace(bounds[param_index][0], bounds[param_index][1], n_grid)
    ll_values = np.empty_like(p_grid)
    ll_mle = loglikelihood(theta_mle, method)

    nuisance_guess = [theta_mle[i] for i in range(len(theta_mle)) if i != param_index]

    for j, val in enumerate(tqdm(p_grid, desc=f"Param {param_index+1}")):
        theta_fixed = theta_mle.copy()
        theta_fixed[param_index] = val
        free_idx = [i for i in range(len(theta_mle)) if i != param_index]
        free_bounds = [bounds[i] for i in free_idx]

        def neg_ll_nuisance(free_params):
            theta_var = theta_fixed.copy()
            theta_var[free_idx] = free_params
            return -loglikelihood(theta_var, method)

        res = minimize(neg_ll_nuisance, nuisance_guess, bounds=free_bounds,
                       method="L-BFGS-B", options={"ftol":1e-5, "gtol":1e-5, "maxiter":50})
        ll_values[j] = -res.fun
        nuisance_guess = res.x

    norm_ll = ll_values - ll_mle
    return param_index, p_grid, norm_ll

# -----------------------------
# Run profiles only for kinetic parameters (first 6)
# -----------------------------
print("Running profile likelihood analysis for kinetic parameters...")
results = Parallel(n_jobs=4)(
    delayed(profile_likelihood)(i, theta_mle, bounds_list, "BDF", n_grid=15)
    for i in range(6)
)

# -----------------------------
# Extract 95% CIs for kinetic parameters
# -----------------------------
cutoff = -chi2.ppf(0.95, df=1)/2
param_labels = [r"$c_1$", r"$c_{max}$", r"$\phi_e$", r"$\gamma_e$", r"$\delta_e$", r"$\eta_e$"]

def extract_ci(p_grid, norm_ll, cutoff):
    """Interpolate CI from profile likelihood curve"""
    try:
        # Find where the profile crosses the cutoff
        below = norm_ll < cutoff
        if np.any(below):
            # Get indices where it crosses
            cross_idx = np.where(np.diff(below.astype(int)) != 0)[0]
            if len(cross_idx) >= 2:
                lower = np.interp(cutoff, norm_ll[cross_idx[0]:cross_idx[0]+2],
                                 p_grid[cross_idx[0]:cross_idx[0]+2])
                upper = np.interp(cutoff, norm_ll[cross_idx[1]:cross_idx[1]+2],
                                 p_grid[cross_idx[1]:cross_idx[1]+2])
                return lower, upper
        return None, None
    except:
        return None, None

cis = {}
for param_index, p_grid, norm_ll in results:
    lower, upper = extract_ci(p_grid, norm_ll, cutoff)
    cis[param_labels[param_index]] = (lower, upper)

print("\n95% Confidence Intervals (from profiles):")
for k,v in cis.items():
    print(f"{k}: {v}")

# -----------------------------
# Parameter co-variability analysis (only for kinetic parameters)
# -----------------------------
def parameter_covariability(param_indices, theta_mle, bounds, method="BDF", n_grid=8):
    """Analyze co-variability between two parameters"""
    param1_idx, param2_idx = param_indices
    p1_grid = np.linspace(bounds[param1_idx][0], bounds[param1_idx][1], n_grid)
    p2_grid = np.linspace(bounds[param2_idx][0], bounds[param2_idx][1], n_grid)

    # Create meshgrid for the two parameters
    P1, P2 = np.meshgrid(p1_grid, p2_grid)
    ll_values = np.zeros_like(P1)

    ll_mle = loglikelihood(theta_mle, method)
    cutoff = -chi2.ppf(0.95, df=2)/2  # For 2D confidence region

    # For each point in the grid, optimize the remaining parameters
    for i in tqdm(range(n_grid), desc=f"Params {param1_idx+1}-{param2_idx+1}"):
        for j in range(n_grid):
            theta_fixed = theta_mle.copy()
            theta_fixed[param1_idx] = P1[i, j]
            theta_fixed[param2_idx] = P2[i, j]

            free_indices = [idx for idx in range(len(theta_mle))
                          if idx not in [param1_idx, param2_idx]]
            free_bounds = [bounds[idx] for idx in free_indices]

            def neg_ll_nuisance(free_params):
                theta_var = theta_fixed.copy()
                theta_var[free_indices] = free_params
                return -loglikelihood(theta_var, method)

            # Use the MLE values as initial guess for nuisance parameters
            nuisance_guess = theta_mle[free_indices]
            res = minimize(neg_ll_nuisance, nuisance_guess, bounds=free_bounds,
                          method="L-BFGS-B", options={"ftol":1e-4, "maxiter":30})

            ll_values[i, j] = -res.fun

    norm_ll = ll_values - ll_mle
    return param1_idx, param2_idx, P1, P2, norm_ll, cutoff

# -----------------------------
# Run co-variability analysis for parameter pairs (only kinetic parameters)
# -----------------------------
# Select which parameter pairs to analyze (first 6 parameters)
param_pairs = list(itertools.combinations(range(6), 2))[:6]  # Only first 6 pairs to save time
covariability_results = []

print("Running parameter co-variability analysis for kinetic parameters...")
for param_pair in param_pairs:
    result = parameter_covariability(param_pair, theta_mle, bounds_list, "BDF", n_grid=6)
    covariability_results.append(result)
    print(f"Completed analysis for parameters {param_pair[0]+1} and {param_pair[1]+1}")

# -----------------------------
# Plot co-variability results
# -----------------------------
param_labels = [r"$a_1$", r"$a_2$", r"$a_3$", r"$a_4$", r"$a_5$", r"$a_6$"]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, (param1_idx, param2_idx, P1, P2, norm_ll, cutoff) in enumerate(covariability_results):
    ax = axes[idx]

    # Plot contour of the confidence region
    contour = ax.contour(P1, P2, norm_ll, levels=[cutoff], colors='red', linewidths=2)
    ax.clabel(contour, inline=True, fontsize=10)

    # Add a heatmap for the log-likelihood surface
    im = ax.contourf(P1, P2, norm_ll, levels=10, alpha=0.7, cmap='viridis')

    # Mark the MLE point
    ax.plot(theta_mle[param1_idx], theta_mle[param2_idx], 'ro', markersize=8)

    ax.set_xlabel(param_labels[param1_idx])
    ax.set_ylabel(param_labels[param2_idx])
    ax.set_title(f"{param_labels[param1_idx]} vs {param_labels[param2_idx]}")

    # Add colorbar
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.suptitle("Parameter Co-variability in 95% Confidence Region (Kinetic Parameters Only)", fontsize=14)
plt.subplots_adjust(top=0.93)
plt.show()

# -----------------------------
# Extract parameter correlations
# -----------------------------
print("\nParameter correlations from co-variability analysis:")
for param1_idx, param2_idx, P1, P2, norm_ll, cutoff in covariability_results:
    # Find points within the confidence region
    in_region = norm_ll >= cutoff

    if np.any(in_region):
        # Calculate correlation coefficient for points in confidence region
        p1_values = P1[in_region]
        p2_values = P2[in_region]
        correlation = np.corrcoef(p1_values, p2_values)[0, 1]

        print(f"{param_labels[param1_idx]} & {param_labels[param2_idx]}: r = {correlation:.3f}")

        # Calculate parameter ranges within confidence region
        p1_min, p1_max = np.min(p1_values), np.max(p1_values)
        p2_min, p2_max = np.min(p2_values), np.max(p2_values)

        print(f"  {param_labels[param1_idx]} range in 95% CI: [{p1_min:.3g}, {p1_max:.3g}]")
        print(f"  {param_labels[param2_idx]} range in 95% CI: [{p2_min:.3g}, {p2_max:.3g}]")
    else:
        print(f"{param_labels[param1_idx]} & {param_labels[param2_idx]}: No significant correlation")

print("\nAnalysis complete!")

In [ ]:
param_labels = [r"$c_1$", r"$c_{max}$", r"$\phi_{e}$",r"$\gamma_e$", r"$\delta_{e}$", r"$\eta_{e}$"]
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, (param1_idx, param2_idx, P1, P2, norm_ll, cutoff) in enumerate(covariability_results):
    ax = axes[idx]

    # Plot contour of the confidence region
    contour = ax.contour(P1, P2, norm_ll, levels=[cutoff], colors='orange', linewidths=2)
    ax.clabel(contour, inline=True, fontsize=10)

    # Add a heatmap for the log-likelihood surface
    im = ax.contourf(P1, P2, norm_ll, levels=10, alpha=0.7, cmap='Blues')

    # Mark the MLE point
    ax.plot(theta_mle[param1_idx], theta_mle[param2_idx], 'ro', markersize=8)

    ax.set_xlabel(param_labels[param1_idx])
    ax.set_ylabel(param_labels[param2_idx])
    #ax.set_title(f"{param_labels[param1_idx]} vs {param_labels[param2_idx]}")

    # Add colorbar
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.suptitle("Parameter Co-variability in 95% Confidence Region (Kinetic Parameters Only)", fontsize=14)
plt.subplots_adjust(top=0.93)
plt.show()

# -----------------------------
# Extract parameter correlations
# -----------------------------
print("\nParameter correlations from co-variability analysis:")
for param1_idx, param2_idx, P极, P2, norm_ll, cutoff in covariability_results:
    # Find points within the confidence region
    in_region = norm_ll >= cutoff

    if np.any(in_region):
        # Calculate correlation coefficient for points in confidence region
        p1_values = P1[in_region]
        p2_values = P2[in_region]
        correlation = np.corrcoef(p1_values, p2_values)[0, 1]

        print(f"{param_labels[param1_idx]} & {param_labels[param2_idx]}: r = {correlation:.3f}")

        # Calculate parameter ranges within confidence region
        p1_min, p1_max = np.min(p1_values), np.max(p1_values)
        p2_min, p2_max = np.min(p2_values), np.max(p2_values)

        print(f"  {param_labels[param1_idx]} range in 95% CI: [{p1_min:.3g}, {p1_max:.3g}]")
        print(f"  {param_labels[param2_idx]} range in 95% CI: [{p2_min:.3g}, {p2_max:.3g}]")
    else:
        print(f"{param_labels[param1_idx]} & {param_labels[param2_idx]}: No significant correlation")

print("\nAnalysis complete!")